# Chatbot Intialization

In [ ]:
!pip3 install rasa[full]
!pip3 install nest_asyncio

In [ ]:
# !pip3 uninstall packaging 
!pip3 install packaging==20.9
# !pip3 show packaging
import packaging
packaging.__version__
# from packaging.version import LegacyVersion

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'20.9'

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


In [ ]:
from rasa.cli.scaffold import create_initial_project
import os

project = "test-project"
create_initial_project(project)

# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

['domain.yml', 'credentials.yml', 'actions', 'config.yml', 'tests', 'endpoints.yml', 'data']


In [ ]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"
print(config, training_files, domain, output)

config.yml data/ domain.yml models/


# OpenAI Initialization

In [ ]:
!pip3 install mwclient
!pip3 install mwparserfromhell
!pip3 install openai
!pip3 install tiktoken

In [ ]:
# imports
import mwclient  # for downloading example Wikipedia articles
import mwparserfromhell  # for splitting Wikipedia articles into sections
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search

In [ ]:
import os
import pandas as pd

In [ ]:
!mkdir new_delhi

In [ ]:
directory_path = 'new_delhi/'

In [ ]:
dfs = []

In [ ]:
for file in os.listdir(directory_path):
    if file.endswith('.txt'):
        hotel_name = file.split('india_new delhi_')[-1]
        hotel_name = hotel_name[:-4]
        hotel_name = ' '.join([word.capitalize() for word in hotel_name.split('_')])
        # df = pd.read_csv(directory_path + file)
        df = pd.read_csv(directory_path + file, sep='\t', header=None, index_col=False, names=['Date', 'Review Heading', 'Review'])
        df['City'] = 'New Delhi'
        df['Hotel Name'] = hotel_name
        # print("Hotel : " + hotel_name)
        dfs.append(df)
    else:
        continue
# print(len(dfs))
# df.head()


In [ ]:
final_df = pd.concat(dfs, ignore_index=True)

In [ ]:
final_df.head()

,Date,Review Heading,Review,City,Hotel Name
0,NaN,Good but noisy,We stayed only one night and our room was clos...,New Delhi,Hotel Palace Heights
1,Nov 20 2009,Good Place to Stay,This hotel is fairly small but well sited in t...,New Delhi,Hotel Palace Heights
2,Nov 19 2009,Centrally located clean and with a good restau...,As with other reviews the driver was unsure ex...,New Delhi,Hotel Palace Heights
3,Nov 15 2009,Clean rooms great service good location and value,The staff were very attentive and our room was...,New Delhi,Hotel Palace Heights
4,Nov 13 2009,Took our money and left us stranded on the str...,My husband and I have traveled widely around I...,New Delhi,Hotel Palace Heights


In [ ]:
hotel_reviews = []

In [ ]:
final_df_1 = final_df.dropna(how='any')

print("Updated DataFrame:")
print(len(final_df_1))

Updated DataFrame:
633


In [ ]:
for index, row in final_df_1.iterrows():
    hotel_name = row['Hotel Name']
    hotel_review = row['Review']
    hotel_review_heading = row['Review Heading']
    
    try: 
      review = hotel_name + " : " + hotel_review_heading + " " + hotel_review
    except TypeError: 
      print(hotel_name, hotel_review, hotel_review_heading)
    hotel_reviews.append(review)


In [ ]:
final_df_1.to_csv("hotel_review_df", sep='\t', encoding='utf-8')

In [ ]:
new = final_df_1[['Hotel Name', 'Review Heading', 'Review']].copy()

In [ ]:
new.to_csv("hotel_review_updated", sep='\t', encoding='utf-8')

In [ ]:
print(hotel_reviews[0])

Hotel City Park : Lovely Place The hotel is amazing n i m impressed by d management of staff..Food quality is too good..the hotel is well built n neat &amp; clean place to hang out and for function purposes also


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))
  
def drop_if_more_than_limit(reviews: list[str]):
  max_tokens = 1600
  final_reviews = []
  for s in reviews:
    if num_tokens(s) < max_tokens:
        final_reviews.append(s)
  return final_reviews
final_reviews = drop_if_more_than_limit(hotel_reviews)
print(len(final_reviews))

655


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# calculate embeddings
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
BATCH_SIZE = 1000  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(final_reviews), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = final_reviews[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # double check embeddings are in same order as input
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)

df_embedding = pd.DataFrame({"text": hotel_reviews, "embedding": embeddings})

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Batch 0 to 999


In [ ]:
!mkdir model

mkdir: cannot create directory ‘model’: File exists


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# save document chunks and embeddings

SAVE_PATH = "model/new_delhi_hotels1.csv"

df_embedding.to_csv(SAVE_PATH, index=False)


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Chatbot Train

In [ ]:
!pip3 install rasa

In [ ]:
!rasa init

In [6]:
#!rasa init
!rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1048: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

In [ ]:
import rasa

model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

In [ ]:
from rasa.jupyter import chat

endpoints = "endpoints.yml"
# endpoints = None
chat(model_path.model, endpoints)

AttributeError: ignored

In [ ]:
import shutil
shutil.make_archive('test-project', 'zip')

'test-project.zip'

In [ ]:
!ls

actions     credentials.yml  domain.yml     models	      tests
config.yml  data	     endpoints.yml  test-project.zip


In [ ]:
!pwd

/content/test-project


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# rasa cli

In [ ]:
!rasa run

In [ ]:
!rasa run actions; rasa shell

/usr/local/lib/python3.9/dist-packages/rasa/core/tracker_store.py:1048: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 42

In [ ]:
!rasa shell

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.9/dist-packages/rasa/core/tracker_store.py:1048: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 42